Run this notebook in the same virtual environment with superlinked server
to ensure the same version of `superlinked` framework.

In [1]:
!pip freeze | grep superlinked

superlinked==17.2.2
superlinked-server==0.7.2


In [2]:
from pathlib import Path
import sys

# depending on the user's setup
# we will try to find the superlinked_app directory
# and add it to the sys.path

cwd = Path.cwd()
if cwd.name == "superlinked-recipes":
    project_dir = cwd / "projects" / "hotel-search"
elif cwd.name == "notebooks":
    project_dir = cwd.parent
else:
    project_dir = cwd

superlinked_app_dir = project_dir / "superlinked_app"
assert superlinked_app_dir.exists(), (
    f"{superlinked_app_dir} does not exist\n"
    "are you sure you are in the hotel-search/notebooks directory?"
)

if str(project_dir) not in sys.path:
    sys.path.append(str(project_dir))
    print(f"project_dir is added to sys.path:\n{project_dir}")
else:
    print(f"project_dir is already in sys.path:\n{project_dir}")

project_dir is added to sys.path:
superlinked-recipes/projects/hotel-search


In [3]:
from superlinked import framework as sl

from superlinked_app.index import index, hotel_schema
from superlinked_app.query import query
from superlinked_app.api import vector_database

superlinked-recipes/projects/hotel-search/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
source = sl.InMemorySource(hotel_schema)
executor = sl.InteractiveExecutor(
    sources=[source],
    indices=[index],
    vector_database=vector_database,
)
app = executor.run()

In [5]:
params = {
    "natural_query": "well rated pet friendly hotel in london, cozy interior",
    "limit": 3,
}

result = app.query(query, **params)
print(result.knn_params)
result.to_pandas()

{'price_weight': 0.0, 'rating_weight': 1.0, 'rating_count_weight': 0.0, 'description_weight': 1.0, 'description': 'Hotel with cozy interior.', 'limit': 3, 'city': ['London'], 'min_price': None, 'max_price': None, 'min_rating': None, 'max_rating': None, 'accomodation_types_include': None, 'accomodation_types_exclude': None, 'property_amenities_include_all': None, 'property_amenities_include_any': ['Pets allowed'], 'property_amenities_exclude': None, 'room_amenities_include_all': None, 'room_amenities_include_any': None, 'room_amenities_exclude': None, 'wellness_spa_include_all': None, 'wellness_spa_include_any': None, 'wellness_spa_exclude': None, 'accessibility_include_all': None, 'accessibility_include_any': None, 'accessibility_exclude': None, 'for_children_include_all': None, 'for_children_include_any': None, 'for_children_exclude': None, 'natural_query': 'well rated pet friendly hotel in london, cozy interior', 'system_prompt_param': "Extract the search parameters from the user que

,description,price,rating,rating_count,accomodation_type,property_amenities,room_amenities,wellness_spa,accessibility,for_children,city,country,id,similarity_score,rank
0,Hotel Cafe Royal has been around since 1865 an...,834,9.6,5630,Hotel,"[24-hour reception, 24-hour room service, Bell...","[Air conditioning, Cable TV, Central heating, ...","[Beauty salon, Body treatments, Hammam, Hot st...","[Accessible hotel, Accessible parking, Wheelch...","[Childcare, Cot, Kids' club, Playground]",London,United Kingdom,Hotel Cafe Royal,0.829669,0
1,A stellar renovation of an iconic Edwardian to...,220,9.0,5540,Serviced apartment,"[24-hour reception, Breakfast, Cashless paymen...","[Air conditioning, Cable TV, Central heating, ...",[],"[Accessible hotel, Wheelchair accessible]",[Cot],London,United Kingdom,Town Hall Hotel - Shoreditch,0.822555,1
2,"One of oldest hotel in London, Flemings is a 4...",400,8.8,10176,Hotel,"[24-hour reception, Breakfast, Business centre...","[Air conditioning, Cable TV, Central heating, ...","[Beauty salon, Massage]","[Accessible hotel, Wheelchair accessible]",[],London,United Kingdom,Flemings Mayfair - Small Luxury Hotel of the W...,0.818173,2


### Partial scores

In [6]:
from superlinked.framework.dsl.executor.query.query_executor import (
    QueryExecutor,
    QueryDescriptor,
    QueryParamValueSetter,
    QueryVectorFactory,
)
from superlinked.evaluation.vector_sampler import VectorCollection
from superlinked.framework.dsl.space.space import Space

import numpy as np
import pandas as pd

In [7]:
def get_query_vector(
    index: sl.Index,
    app: sl.InteractiveExecutor,
    query_descriptor: QueryDescriptor,
    params: dict,
):
    query_vector_factory = QueryVectorFactory(index._dag)
    query_executor = QueryExecutor(app, query_descriptor, query_vector_factory)
    query_descriptor = QueryParamValueSetter.set_values(query_descriptor, params)
    query_vector = query_executor._produce_query_vector(query_descriptor)
    return query_vector


def get_partial_scores(
    spaces: list[Space], stored_vectors: VectorCollection, query_vector: np.ndarray
) -> dict[str, dict[str, float]]:

    i_start = 0
    partial_scores = {}
    ids = stored_vectors.id_list

    for space in spaces:
        fields = list(space._field_set)
        assert len(fields) == 1
        name = fields[0].name
        i_end = i_start + space.length
        v_space = stored_vectors.vectors[:, i_start:i_end]
        q_space = query_vector[i_start:i_end]
        partial_scores_space = np.dot(v_space, q_space)

        partial_scores[name] = {id: x for id, x in zip(ids, partial_scores_space)}
        i_start = i_end

    return partial_scores

In [8]:
vector_sampler = sl.VectorSampler(app)

In [9]:
params = {
    "natural_query": "well rated pet friendly hotel in london, cozy interior",
    "limit": 3,
}

query_descriptor = QueryParamValueSetter.set_values(query, params)
knn_params = query_descriptor.calculate_value_by_param_name()

knn_params.pop("natural_query")
knn_params.pop("radius_param")
knn_params.pop("system_prompt_param")

print(knn_params)

{'price_weight': 0.0, 'rating_weight': 1.0, 'rating_count_weight': 0.0, 'description_weight': 1.0, 'description': 'Hotel with cozy interior.', 'limit': 3, 'city': ['London'], 'min_price': None, 'max_price': None, 'min_rating': None, 'max_rating': None, 'accomodation_types_include': None, 'accomodation_types_exclude': None, 'property_amenities_include_all': None, 'property_amenities_include_any': ['Pets allowed'], 'property_amenities_exclude': None, 'room_amenities_include_all': None, 'room_amenities_include_any': None, 'room_amenities_exclude': None, 'wellness_spa_include_all': None, 'wellness_spa_include_any': None, 'wellness_spa_exclude': None, 'accessibility_include_all': None, 'accessibility_include_any': None, 'accessibility_exclude': None, 'for_children_include_all': None, 'for_children_include_any': None, 'for_children_exclude': None, 'similar_description_weight': 1.0}


In [10]:
result = app.query(query, **knn_params)
ids = [e.header.object_id for e in result.entities]

query_vector = get_query_vector(index, app, query, knn_params).value.flatten()

stored_vectors: VectorCollection = vector_sampler.get_vectors_by_ids(
    id_=ids,
    index=index,
    schema=hotel_schema,
)

In [11]:
partial_scores = get_partial_scores(index._spaces, stored_vectors, query_vector)
df_partial_scores = pd.DataFrame(partial_scores)
df_partial_scores

,description,price,rating,rating_count
Hotel Cafe Royal,0.330655,0.0,0.499013,0.0
Town Hall Hotel - Shoreditch,0.328711,0.0,0.493844,0.0
Flemings Mayfair - Small Luxury Hotel of the World,0.327029,0.0,0.491144,0.0
